# Network

In [ ]:
import tensorflow as tf

class Network:
    """
    Build a physics informed neural network (PINN) model for the steady Navier-Stokes equations.
    Attributes:
        activations: custom activation functions.
    """

    def __init__(self):
        """
        Setup custom activation functions.
        """
        self.activations = {
            'tanh' : 'tanh',
            'swish': self.swish,
            'mish' : self.mish,
        }
    # Examples of other activation functions:
    def swish(self, x):
        """
        Swish activation function.
        Args:
            x: activation input.
        Returns:
            Swish output.
        """
        return x * tf.math.sigmoid(x)

    def mish(self, x):
        """
        Mish activation function.
        Args:
            x: activation input.
        Returns:
            Mish output.
        """
        return x * tf.math.tanh(tf.softplus(x))

    def build(self, num_inputs=2, layers=[48,48,48,48], activation='tanh', num_outputs=3):
        """
        Build a PINN model for the steady Navier-Stokes equation with input shape (x,y) and output shape (u, v, p).
        Args:
            num_inputs: number of input variables. Default is 2 for (x, y).
            layers: number of hidden layers.
            activation: activation function in hidden layers.
            num_outpus: number of output variables. Default is 3 for (u, v, p).
        Returns:
            keras network model
        """

        # input layer
        inputs = tf.keras.layers.Input(shape=(num_inputs,))
        # hidden layers
        x = inputs
        for layer in layers:
            x = tf.keras.layers.Dense(layer, activation=self.activations[activation],
                kernel_initializer='he_normal')(x)
        # output layer
        outputs = tf.keras.layers.Dense(num_outputs,
            kernel_initializer='he_normal')(x)

        return tf.keras.models.Model(inputs=inputs, outputs=outputs)

# Layer

In [ ]:
import tensorflow as tf

class GradientLayer(tf.keras.layers.Layer):
    """
    Custom layer to compute derivatives for the steady Navier-Stokes equation.
    Attributes:
        model: keras network model.
    """

    def __init__(self, model, **kwargs):
        """
        Args:
            model: keras network model.
        """

        self.model = model
        super().__init__(**kwargs)

    def call(self, xyt):
        """
        Computing derivatives for the steady Navier-Stokes equation.
        Args:
            xy: input variable.
        Returns:
            psi: stream function.
            p_grads: pressure and its gradients.
            u_grads: u and its gradients.
            v_grads: v and its gradients.
        """

        x, y = [ xyt[..., i, tf.newaxis] for i in range(xyt.shape[-1]) ]
        with tf.GradientTape(persistent=True) as ggg:
            ggg.watch(x)
            ggg.watch(y)

            with tf.GradientTape(persistent=True) as gg:
                gg.watch(x)
                gg.watch(y)

                with tf.GradientTape(persistent=True) as g:
                    g.watch(x)
                    g.watch(y)

                    u_v_p = self.model(tf.concat([x, y], axis=-1))
                    u = u_v_p[..., 0, tf.newaxis]
                    v = u_v_p[..., 1, tf.newaxis]
                    p = u_v_p[..., 2, tf.newaxis]
                u_x = g.batch_jacobian(u, x)[..., 0]
                v_x = g.batch_jacobian(v, x)[..., 0]
                u_y = g.batch_jacobian(u, y)[..., 0]
                v_y = g.batch_jacobian(v, y)[..., 0]
                p_x = g.batch_jacobian(p, x)[..., 0]
                p_y = g.batch_jacobian(p, y)[..., 0]

                del g
            u_xx = gg.batch_jacobian(u_x, x)[..., 0]
            u_yy = gg.batch_jacobian(u_y, y)[..., 0]

            v_xx = gg.batch_jacobian(v_x, x)[..., 0]
            v_yy = gg.batch_jacobian(v_y, y)[..., 0]

            del gg
        # if more derivatives are required...
        del ggg

        p_grads = p, p_x, p_y
        u_grads = u, u_x, u_y, u_xx, u_yy
        v_grads = v, v_x, v_y, v_xx, v_yy

        return p_grads, u_grads, v_grads

# Optimizer

In [ ]:
import scipy.optimize
import numpy as np
import tensorflow as tf

class L_BFGS_B:
    """
    Optimize the keras network model using L-BFGS-B algorithm.
    Attributes:
        model: optimization target model.
        samples: training samples.
        factr: function convergence condition. typical values for factr are: 1e12 for low accuracy;
               1e7 for moderate accuracy; 10.0 for extremely high accuracy.
        pgtol: gradient convergence condition.
        m: maximum number of variable metric corrections used to define the limited memory matrix.
        maxls: maximum number of line search steps (per iteration).
        maxiter: maximum number of iterations.
        metris: log metrics
        progbar: progress bar
    """

    def __init__(self, model, x_train, y_train, factr=1e5, m=50, maxls=50, maxiter=30000):
        """
        Args:
            model: optimization target model.
            samples: training samples.
            factr: convergence condition. typical values for factr are: 1e12 for low accuracy;
                   1e7 for moderate accuracy; 10.0 for extremely high accuracy.
            pgtol: gradient convergence condition.
            m: maximum number of variable metric corrections used to define the limited memory matrix.
            maxls: maximum number of line search steps (per iteration).
            maxiter: maximum number of iterations.
        """

        # set attributes
        self.model = model
        self.x_train = [ tf.constant(x, dtype=tf.float32) for x in x_train ]
        self.y_train = [ tf.constant(y, dtype=tf.float32) for y in y_train ]
        self.factr = factr
        self.m = m
        self.maxls = maxls
        self.maxiter = maxiter
        self.metrics = ['loss']
        # initialize the progress bar
        self.progbar = tf.keras.callbacks.ProgbarLogger(
            count_mode='steps', stateful_metrics=self.metrics)
        self.progbar.set_params( {
            'verbose':1, 'epochs':1, 'steps':self.maxiter, 'metrics':self.metrics})

    def set_weights(self, flat_weights):
        """
        Set weights to the model.
        Args:
            flat_weights: flatten weights.
        """

        # get model weights
        shapes = [ w.shape for w in self.model.get_weights() ]
        # compute splitting indices
        split_ids = np.cumsum([ np.prod(shape) for shape in [0] + shapes ])
        # reshape weights
        weights = [ flat_weights[from_id:to_id].reshape(shape)
            for from_id, to_id, shape in zip(split_ids[:-1], split_ids[1:], shapes) ]
        # set weights to the model
        self.model.set_weights(weights)

    @tf.function
    def tf_evaluate(self, x, y):
        """
        Evaluate loss and gradients for weights as tf.Tensor.
        Args:
            x: input data.
        Returns:
            loss and gradients for weights as tf.Tensor.
        """

        with tf.GradientTape() as g:
            loss = tf.reduce_mean(tf.keras.losses.logcosh(self.model(x), y))
        grads = g.gradient(loss, self.model.trainable_variables)
        return loss, grads

    def evaluate(self, weights):
        """
        Evaluate loss and gradients for weights as ndarray.
        Args:
            weights: flatten weights.
        Returns:
            loss and gradients for weights as ndarray.
        """

        # update weights
        self.set_weights(weights)
        # compute loss and gradients for weights
        loss, grads = self.tf_evaluate(self.x_train, self.y_train)
        # convert tf.Tensor to flatten ndarray
        loss = loss.numpy().astype('float64')
        grads = np.concatenate([ g.numpy().flatten() for g in grads ]).astype('float64')

        return loss, grads

    def callback(self, weights):
        """
        Callback that prints the progress to stdout.
        Args:
            weights: flatten weights.
        """
        self.progbar.on_batch_begin(0)
        loss, _ = self.evaluate(weights)
        self.progbar.on_batch_end(0, logs=dict(zip(self.metrics, [loss])))

    def fit(self):
        """
        Train the model using L-BFGS-B algorithm.
        """

        # get initial weights as a flat vector
        initial_weights = np.concatenate(
            [ w.flatten() for w in self.model.get_weights() ])
        # optimize the weight vector
        print('Optimizer: L-BFGS-B (maxiter={})'.format(self.maxiter))
        self.progbar.on_train_begin()
        self.progbar.on_epoch_begin(1)
        scipy.optimize.fmin_l_bfgs_b(func=self.evaluate, x0=initial_weights,
            factr=self.factr, m=self.m,
            maxls=self.maxls, maxiter=self.maxiter, callback=self.callback)
        self.progbar.on_epoch_end(1)
        self.progbar.on_train_end()

# PINN

In [ ]:
import tensorflow as tf
import numpy as np

class PINN:
    """
    Build a physics informed neural network (PINN) model for the steady Navier-Stokes equation.
    Attributes:
        network: keras network model with input (x, y) and output (u, v, p).
        rho: density.
        nu: viscosity.
        grads: gradient layer.
    """

    def __init__(self, network, rho=1, mu=0.01):
        """
        Args:
            network: keras network model with input (x, y) and output (u, v, p).
            rho: density.
            nu: viscosity.
        """

        self.network = network
        self.rho = rho
        self.mu = mu
        self.grads = GradientLayer(self.network)

    def build(self):
        """
        Build a PINN model for the steady Navier-Stokes equation.
        Returns:
            PINN model for the steady Navier-Stokes equation with
                input: [ (x, y) relative to equation,
                         (x, y) relative to boundary condition ],
                output: [ (u, v) relative to equation (must be zero),
                          (psi, psi) relative to boundary condition (psi is duplicated because outputs require the same dimensions),
                          (u, v) relative to boundary condition ]
        """

        # equation input: (x, y)
        xy_eqn = tf.keras.layers.Input(shape=(2,))
        # boundary condition
        xy_in = tf.keras.layers.Input(shape=(2,))
        xy_out = tf.keras.layers.Input(shape=(2,))
        xy_w1 = tf.keras.layers.Input(shape=(2,))
        xy_w2 = tf.keras.layers.Input(shape=(2,))
        xy_circle = tf.keras.layers.Input(shape=(2,))

        # compute gradients relative to equation
        p_grads, u_grads, v_grads = self.grads(xy_eqn)
        _, p_x, p_y = p_grads
        u, u_x, u_y, u_xx, u_yy = u_grads
        v, v_x, v_y, v_xx, v_yy = v_grads
        # compute equation loss
        u_eqn =  u*u_x + v*u_y + p_x/self.rho - self.mu*(u_xx + u_yy) / self.rho
        v_eqn =  u*v_x + v*v_y + p_y/self.rho - self.mu*(v_xx + v_yy) / self.rho
        uv_eqn = u_x + v_y
        uv_eqn = tf.concat([u_eqn, v_eqn, uv_eqn], axis=-1)

        # compute gradients relative to boundary condition
        p_r, u_grads_r, v_grads_r = self.grads(xy_out)
        uv_out = tf.concat([p_r[0], p_r[0], p_r[0]], axis=-1)

        p_l, u_grads_l, v_grads_l = self.grads(xy_w1)
        uv_w1 = tf.concat([u_grads_l[0], v_grads_l[0], p_l[2]], axis=-1)
        
        p_l, u_grads_l, v_grads_l = self.grads(xy_w2)
        uv_w2 = tf.concat([u_grads_l[0], v_grads_l[0], p_l[2]], axis=-1)
        
        p_l, u_grads_l, v_grads_l = self.grads(xy_circle)
        uv_circle = tf.concat([u_grads_l[0], v_grads_l[0], u_grads_l[0]], axis=-1)

        p_inn, u_inn, v_inn = self.grads(xy_in)
        uv_in = tf.concat([u_inn[0], v_inn[0], u_inn[0]], axis=-1)

        # build the PINN model for the steady Navier-Stokes equation
        return tf.keras.models.Model(
            inputs=[xy_eqn, xy_w1, xy_w2, xy_out, xy_in, xy_circle], outputs=[uv_eqn, uv_in, uv_out, uv_w1, uv_w2, uv_circle])

In [3]:
# Main

In [ ]:
import tf_silent
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
from matplotlib.gridspec import GridSpec
from pinn import PINN
from network import Network
from optimizer import L_BFGS_B



def mass_cons(network, xy):
    """
    Compute u_x and v_y
    Args:
        xy: network input variables as ndarray.
    Returns:
        (u_x, v_y) as ndarray.
    """

    xy = tf.constant(xy)
    x, y = [ xy[..., i, tf.newaxis] for i in range(xy.shape[-1]) ]
    with tf.GradientTape(persistent=True) as g:
      g.watch(x)
      g.watch(y)

      u_v_p = network(tf.concat([x, y], axis=-1))
      u = u_v_p[..., 0, tf.newaxis]
      v = u_v_p[..., 1, tf.newaxis]
      p = u_v_p[..., 2, tf.newaxis]
    u_x = g.batch_jacobian(u, x)[..., 0]
    v_y = g.batch_jacobian(v, y)[..., 0]

    return u_x.numpy(), v_y.numpy()

def u_0(xy):
    """
    Initial wave form.
    Args:
        tx: variables (t, x) as tf.Tensor.
    Returns:
        u(t, x) as tf.Tensor.
    """

    x = xy[..., 0, None]
    y = xy[..., 1, None]


    return    4*y*(1 - y) 


def contour(x, y, z, title, levels=100):
    """
    Contour plot.
    Args:
        grid: plot position.
        x: x-array.
        y: y-array.
        z: z-array.
        title: title string.
        levels: number of contour lines.
    """

    # get the value range
    vmin = np.min(z)
    vmax = np.max(z)
    # plot a contour
    font1 = {'family':'serif','size':20}
    plt.contour(x, y, z, colors='k', linewidths=0.2, levels=levels)
    plt.contourf(x, y, z, cmap='rainbow', levels=levels, norm=Normalize(vmin=vmin, vmax=vmax))
    plt.axes()
    circle = plt.Circle((0.5,0.5),0.1, fc='black')
    plt.gca().add_patch(circle)
    plt.axis('scaled')
    plt.title(title, fontdict = font1)
    plt.xlabel("x", fontdict = font1)
    plt.ylabel("y", fontdict = font1)
    plt.tick_params(axis='both', which='major', labelsize=15)

    cbar = plt.colorbar(pad=0.03, aspect=25, format='%.0e')
    cbar.mappable.set_clim(vmin, vmax)
    cbar.ax.tick_params(labelsize=15)

if __name__ == '__main__':
    """
    Test the physics informed neural network (PINN) model
    for the cavity flow governed by the steady Navier-Stokes equation.
    """

    # number of training samples
    num_train_samples = 5000
    # number of test samples
    num_test_samples = 200

    # inlet flow velocity
    u0 = 1
    # density
    rho = 1
    # viscosity
    mu = 1e-1
    # Re = rho/mu

    # build a core network model
    network = Network().build()
    network.summary()
    # build a PINN model
    pinn = PINN(network, rho=rho, mu=mu).build()

    # Domain and circle data
    x_f =2
    x_ini=0
    y_f=1
    y_ini=0
    Cx = 0.5
    Cy = 0.5
    a = 0.1
    b = 0.1

    xyt_circle = np.random.rand(num_train_samples, 2)
    xyt_circle[...,0] = 2*(a)*xyt_circle[...,0] +(Cx-a)
    xyt_circle[0:num_train_samples//2,1] = b*(1 - (xyt_circle[0:num_train_samples//2,0]-Cx)**2 / a**2)**0.5 + Cy
    xyt_circle[num_train_samples//2:,1] = -b*(1 - (xyt_circle[num_train_samples//2:,0]-Cx)**2 / a**2)**0.5 + Cy

    # create training input
    xyt_eqn = np.random.rand(num_train_samples, 2)
    xyt_eqn[...,0] = (x_f - x_ini)*xyt_eqn[...,0] + x_ini
    xyt_eqn[...,1] = (y_f - y_ini)*xyt_eqn[...,1] + y_ini

    for i in range(num_train_samples):
      while (xyt_eqn[i, 0] - Cx)**2/a**2 + (xyt_eqn[i, 1] - Cy)**2/b**2 < 1:
        xyt_eqn[i, 0] = (x_f - x_ini) * np.random.rand(1, 1) + x_ini
        xyt_eqn[i, 1] = (y_f - y_ini) * np.random.rand(1, 1) + y_ini

    xyt_w1 = np.random.rand(num_train_samples, 2)  # top-bottom boundaries
    xyt_w1[..., 0] = (x_f - x_ini)*xyt_w1[...,0] + x_ini
    xyt_w1[..., 1] =  y_ini          # y-position is 0 or 1

    xyt_w2 = np.random.rand(num_train_samples, 2)  # top-bottom boundaries
    xyt_w2[..., 0] = (x_f - x_ini)*xyt_w2[...,0] + x_ini
    xyt_w2[..., 1] =  y_f

    xyt_out = np.random.rand(num_train_samples, 2)  # left-right boundaries
    xyt_out[..., 0] = x_f

    xyt_in = np.random.rand(num_train_samples, 2)
    xyt_in[...,0] = x_ini

    x_train = [xyt_eqn, xyt_w1, xyt_w2, xyt_out, xyt_in, xyt_circle]

    # create training output
    zeros = np.zeros((num_train_samples, 3))
    #uv_bnd[..., 0] = -u0 * np.floor(xy_bnd[..., 0]) +1
    #ones = np.ones((num_train_samples, 3))
    #onze = np.random.rand(num_train_samples, 3)
    #onze[...,0] = u0
    #onze[...,1] = 0
    #onze[...,2] = u0
    a = u_0(tf.constant(xyt_in)).numpy()
    b = np.zeros((num_train_samples, 1))
    onze = np.random.permutation(np.concatenate([a,b,a],axis = -1))

    y_train = [zeros, onze, zeros, zeros, zeros, zeros]

    # train the model using L-BFGS-B algorithm
    lbfgs = L_BFGS_B(model=pinn, x_train=x_train, y_train=y_train)
    lbfgs.fit()

    # create meshgrid coordinates (x, y) for test plots    

    x = np.linspace(x_ini, x_f, num_test_samples)
    y = np.linspace(y_ini, y_f, num_test_samples)
    x, y = np.meshgrid(x, y)
    xy = np.stack([x.flatten(), y.flatten()], axis=-1)
    # predict (psi, p)
    u_v_p = network.predict(xy, batch_size=len(xy))
    u, v, p = [ u_v_p[..., i].reshape(x.shape) for i in range(u_v_p.shape[-1]) ]
    # compute (u, v)
    u = u.reshape(x.shape)
    v = v.reshape(x.shape)
    p = p.reshape(x.shape)
    # plot test results
    fig = plt.figure(figsize=(16, 8))
    contour(x, y, p, 'p')
    plt.tight_layout()
    plt.show()

    fig = plt.figure(figsize=(16, 8))
    contour(x, y, u, 'u')
    plt.tight_layout()
    plt.show()

    fig = plt.figure(figsize=(16, 8))
    contour(x, y, v, 'v')
    plt.tight_layout()
    plt.show()
    

    ###########################
    from matplotlib.patches import Circle
    font1 = {'family':'serif','size':20}

    fig0, ax0 = plt.subplots(1, 1,figsize=(20,8))
    cf0 = ax0.contourf(x, y, p, np.arange(-0.2, 1, .02),
                   extend='both',cmap='rainbow')
    cbar0 = plt.colorbar(cf0, pad=0.03, aspect=25, format='%.0e')
    plt.title("p", fontdict = font1)
    plt.xlabel("x", fontdict = font1)
    plt.ylabel("y", fontdict = font1)
    ax0.add_patch(Circle((0.5, 0.5), 0.1,color="black"))
    plt.tick_params(axis='both', which='major', labelsize=15)
    cbar0.ax.tick_params(labelsize=15)
    plt.show()

    ###########################

    fig0, ax0 = plt.subplots(1, 1, figsize=(20,8))
    cf0 = ax0.contourf(x, y, u, np.arange(-0.5, 1.1, .02),
                   extend='both',cmap='rainbow')
    cbar0 = plt.colorbar(cf0, )
    plt.title("u", fontdict = font1)
    plt.xlabel("x", fontdict = font1)
    plt.ylabel("y", fontdict = font1)
    ax0.add_patch(Circle((0.5, 0.5), 0.1,color="black"))
    plt.tick_params(axis='both', which='major', labelsize=15)
    cbar0.ax.tick_params(labelsize=15)
    plt.show()

    ###########################

    fig0, ax0 = plt.subplots(1, 1,figsize=(20,8))
    cf0 = ax0.contourf(x, y, v, np.arange(-0.4, 0.4, .02),
                   extend='both',cmap='rainbow')
    cbar0 = plt.colorbar(cf0, pad=0.03, aspect=25, format='%.0e')
    plt.title("v", fontdict = font1)
    plt.xlabel("x", fontdict = font1)
    plt.ylabel("y", fontdict = font1)
    ax0.add_patch(Circle((0.5, 0.5), 0.1,color="black"))
    plt.tick_params(axis='both', which='major', labelsize=15)
    cbar0.ax.tick_params(labelsize=15)
    plt.show()

    ############################ 

    x = np.linspace(0.3, 1, num_test_samples)
    y = np.linspace(0.3, 0.7, num_test_samples)
    x, y = np.meshgrid(x, y)
    xy = np.stack([x.flatten(), y.flatten()], axis=-1)
    # predict (psi, p)
    u_v_p = network.predict(xy, batch_size=len(xy))
    u, v, p = [ u_v_p[..., i].reshape(x.shape) for i in range(u_v_p.shape[-1]) ]
    # compute (u, v)
    u = u.reshape(x.shape)
    v = v.reshape(x.shape)
    p = p.reshape(x.shape)
    # plot test results
    
    fig = plt.figure(figsize=(15, 8))
    #contour(gs[0, 0], x, y, psi, 'psi')
    contour(x, y, p, 'p')
    plt.tight_layout()
    plt.show()

    fig = plt.figure(figsize=(15, 8))
    contour(x, y, u, 'u')
    plt.tight_layout()
    plt.show()

    fig = plt.figure(figsize=(15, 8))
    contour(x, y, v, 'v')
    plt.tight_layout()
    plt.show()

    ###########################
    from matplotlib.patches import Circle
    font1 = {'family':'serif','size':20}

    fig0, ax0 = plt.subplots(1, 1,figsize=(18,8))
    cf0 = ax0.contourf(x, y, p, np.arange(-0.2, 0.6, .02),
                   extend='both',cmap='rainbow')
    cbar0 = plt.colorbar(cf0, pad=0.03, aspect=25, format='%.0e')
    plt.title("p", fontdict = font1)
    plt.xlabel("x", fontdict = font1)
    plt.ylabel("y", fontdict = font1)
    ax0.add_patch(Circle((0.5, 0.5), 0.1,color="black"))
    plt.tick_params(axis='both', which='major', labelsize=15)
    cbar0.ax.tick_params(labelsize=15)
    plt.show()

    ###########################

    fig0, ax0 = plt.subplots(1, 1, figsize=(18,8))
    cf0 = ax0.contourf(x, y, u, np.arange(-0.5, 1.1, .02),
                   extend='both',cmap='rainbow')
    cbar0 = plt.colorbar(cf0, )
    plt.title("u", fontdict = font1)
    plt.xlabel("x", fontdict = font1)
    plt.ylabel("y", fontdict = font1)
    ax0.add_patch(Circle((0.5, 0.5), 0.1,color="black"))
    plt.tick_params(axis='both', which='major', labelsize=15)
    cbar0.ax.tick_params(labelsize=15)
    plt.show()

    ###########################

    fig0, ax0 = plt.subplots(1, 1,figsize=(18,8))
    cf0 = ax0.contourf(x, y, v, np.arange(-0.4, 0.4, .02),
                   extend='both',cmap='rainbow')
    cbar0 = plt.colorbar(cf0, pad=0.03, aspect=25, format='%.0e')
    plt.title("v", fontdict = font1)
    plt.xlabel("x", fontdict = font1)
    plt.ylabel("y", fontdict = font1)
    ax0.add_patch(Circle((0.5, 0.5), 0.1,color="black"))
    plt.tick_params(axis='both', which='major', labelsize=15)
    cbar0.ax.tick_params(labelsize=15)
    plt.show()